# gzip - Soporte de ficheros gzip

Este módulo proporciona una interfaz simple para comprimir y descomprimir
ficheros, de la misma forma que los programas GNU `gzip` o `gunzip`. El algoritmo de
compresión es el mismo que usa la librería `zlib`.

EL módulo proporciona una clase, `GzipFile`, y varias funciones convenientes,
como  `open()`, `compress()` y `decompress()`.

La clase `GzipFile` puede leer
y escribir ficheros comprimidos en formato gzip, realizando las tareas de
compresión y descompresión de forma automática, por lo que para nosotros es
como si fuera un fichero normal y corriente.

Al contrario que con el formato ZIP, el formato gzip solo
permite comprimir y descomprimir un fichero, porque no tiene capacidad
de archivado (Es decir, la posibilidad de añadir varios ficheros
dentro del archivo).

Para crear un fichero comprimido con gzip, simplemente
lo abrimos con el modo `wb` (`w` para escritura, *write*, `b`
para binario, *binary*):

In [12]:
import gzip
   
content = "Hay mucho contenido aquí.\n".encode('utf-8') * 888
with gzip.open('file.txt.gz', 'wb') as f:
    f.write(content)

In [11]:
!cat file.txt.gz

�0 �� ��臄�B��A��+��c�-�e�־/��3OE)��RJ)��RJ)��RJ)��RJ)��RJ)��RJ)��RJ)��RJ)��RJ)��RJ)��RJ)��X��.i�]  

Un ejemplo  de como leer un fichero comprimido::

In [13]:
import gzip
    
with gzip.open('file.txt.gz', 'rb') as f:
    file_content = f.read().decode('utf-8')
print(file_content[0:100])

Hay mucho contenido aquí.
Hay mucho contenido aquí.
Hay mucho contenido aquí.
Hay mucho contenido aq


#### Ejercicio: Comprimir un archivo existente

Comprimir el contenido del fichero `lorem.txt` en un nuevo
archivo `lorem.txt.gz`.

Nota: Los objetos de tipo `GzipFile` incluyen un método `writelines()` que se puede
usar para escribir una lista o secuencia de líneas de texto.

In [17]:
import gzip

with open("lorem.txt", "r") as entrada:
    with gzip.open("lorem.txt.gz", "wb") as salida:
        datos = entrada.read().encode('utf-8')
        salida.write(datos)
!dir -ls lorem.txt lorem.txt.gz

56 -rw-rw-r-- 1 jileon jileon 57060 abr 12 11:36 lorem.txt
24 -rw-rw-r-- 1 jileon jileon 22316 abr 14 19:42 lorem.txt.gz


Usa el siguiente script para comprobar la diferencia de tamaño entre los dos
ficheros.

In [20]:
import glob
import os

for filename in glob.glob("lorem*"):
    size = os.path.getsize(filename)
    print(f"- El fichero {filename} pesa {size} bytes")


- El fichero lorem.txt pesa 57060 bytes
- El fichero lorem.txt.gz pesa 22316 bytes


In [29]:
# Solución

import gzip

with open('lorem.txt', 'rb') as f_in:
    with gzip.open('lorem.txt.gz', 'wb') as f_out: 
        f_out.writelines(f_in.readlines())

In [21]:
with gzip.open('lorem.txt.gz', 'r') as f_in:
    for i, line in enumerate(f_in.readlines()):
        line = line.strip()
        if not line:
            continue
        print(line.decode('utf-8')[0:64])
        if i == 10:
            break

1. Una reunión muy esperada
Cuando el señor Bilbo Bolsón de Bolsón Cerrado anunció que muy p
—Tendrá que pagar —decían—. ¡No es natural, y traerá problemas!
Pero tales problemas no habían llegado y como el señor Bolsón er
El primo mayor y el favorito de Bilbo, era el joven Frodo Bolsón
Pasaron doce años más. Los Bolsón habían dado siempre bulliciosa
